# Step 0: Load Libraries

In [1]:
import pandas as pd
import json
import requests
import csv
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk import FreqDist

import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_sm")

import pickle as pk

import os
from pathlib import Path


folderpath = Path(os.getcwd()).absolute()
data_folderpath = folderpath.parent.joinpath("Data")
amazon_model_folderpath = folderpath.parent.joinpath("SentimentModels").joinpath("Inference").joinpath("Full_Amazon")
shopee_model_folderpath = folderpath.parent.joinpath("SentimentModels").joinpath("Inference").joinpath("Shopee_Aspect")
wordnet_folderpath = folderpath.parent.joinpath("WordNet")

print(folderpath)
print(data_folderpath)
print(amazon_model_folderpath)
print(shopee_model_folderpath)
print(wordnet_folderpath)

C:\Users\onnwe\Desktop\ISS_PLP_Project\Sentiment Analysis with Aspect
C:\Users\onnwe\Desktop\ISS_PLP_Project\Data
C:\Users\onnwe\Desktop\ISS_PLP_Project\SentimentModels\Inference\Full_Amazon
C:\Users\onnwe\Desktop\ISS_PLP_Project\SentimentModels\Inference\Shopee_Aspect
C:\Users\onnwe\Desktop\ISS_PLP_Project\WordNet


# Step 1: Reading Data

In [2]:
filename = "meta_AMAZON_FASHION.json"

filepath = data_folderpath.joinpath(filename)
meta = pd.read_json(filepath, lines=True)

In [3]:
meta

,title,brand,feature,rank,date,asin,imageURL,imageURLHighRes,description,price,also_view,also_buy,fit,details,similar_item,tech1
0,Slime Time Fall Fest [With CDROM and Collector...,Group Publishing (CO),[Product Dimensions:\n \n8....,"13,052,976inClothing,Shoesamp;Jewelry(",8.70 inches,0764443682,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,XCC Qi promise new spider snake preparing men'...,NaN,NaN,"11,654,581inClothing,Shoesamp;Jewelry(",5 star,1291691480,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Magical Things I Really Do Do Too!,Christopher Manos,[Package Dimensions:\n \n8....,"19,308,073inClothing,ShoesJewelry(",5 star,1940280001,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,[For the professional or amateur magician. Ro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Ashes to Ashes, Oranges to Oranges",Flickerlamp Publishing,[Package Dimensions:\n \n8....,"19,734,184inClothing,ShoesJewelry(",5 star,1940735033,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aether & Empire #1 - 2016 First Printing Comic...,NaN,[Package Dimensions:\n \n10...,"10,558,646inClothing,Shoesamp;Jewelry(",5 star,1940967805,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,$4.50,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186632,JT Women's Elegant Off Shoulder Chiffon Maxi L...,JT,NaN,"9,835,890inClothing,ShoesJewelry(",5 star,B01HJGXL4O,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186633,Microcosm Retro Vintage Black Crochet Lace One...,Microcosm,[Package Dimensions:\n \n7....,"11,390,771inClothing,ShoesJewelry(",5 star5 star (0%),B01HJHF97K,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186634,Lookatool Classic Plain Vintage Army Military ...,Lookatool,"[Cotton+Polyester, Imported, Item type:Basebal...","972,275inClothing,ShoesJewelry(",5 star,B01HJGJ9LS,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,$8.53,"[B00XLECZMS, B0018MQAOY, B00N833I4Q, B074DQSPP...","[B07BHQ1FXL, B00XLECZMS, B07CJWM5WY, B07CS97C1...","class=""a-normal a-align-center a-spacing-smal...",NaN,NaN,NaN
186635,Edith Windsor Women's Deep V-neck Beaded Sequi...,Edith Windsor,[Product Dimensions:\n \n9....,"1,964,585inClothing,ShoesJewelry(",5 star,B01HJHTH5U,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,[B077ZLGMJ3],NaN,NaN,NaN,NaN


In [4]:
chunksize = 10000

filename = 'AMAZON_FASHION.json'
filepath = data_folderpath.joinpath(filename)

iter = pd.read_json(filepath, chunksize = chunksize, lines = True)

In [5]:
isFirstChunk = True
count = 0

for chunk in iter:
    #print(count)
    if isFirstChunk:
        chunk.to_json("Sample_Review", orient='records', lines=True)
        review_with_meta = pd.merge(chunk, meta, how="left", on="asin")
        review_with_meta["reviewText"] = review_with_meta["reviewText"].replace('\"',' ', regex=True).str.encode('ascii', 'ignore').str.decode('ascii')
        isFirstChunk = False
    else:
        chunk.to_json("Sample_Review", orient='records', lines=True)
        additional_review_with_meta = pd.merge(chunk, meta, how="left", on="asin")
        additional_review_with_meta["reviewText"] = additional_review_with_meta["reviewText"].replace('\"',' ', regex=True).str.encode('ascii', 'ignore').str.decode('ascii')
        review_with_meta = review_with_meta.append(additional_review_with_meta)
    count += 1

In [6]:
review_with_meta

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,...,imageURL,imageURLHighRes,description,price,also_view,also_buy,fit,details,similar_item,tech1
0,5,True,"10 20, 2014",A1D4G1SNUZWQOT,7106116521,Tracy,Exactly what I needed.,perfect replacements!!,1413763200,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,True,"09 28, 2014",A3DDWDH9PX2YX2,7106116521,Sonja Lau,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",1411862400,3.0,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,False,"08 25, 2014",A2MWC41EW7XL15,7106116521,Kathleen,Love these... I am going to order another pack...,My New 'Friends' !!,1408924800,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,True,"08 24, 2014",A2UH2QQ275NV45,7106116521,Jodi Stoner,too tiny an opening,Two Stars,1408838400,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,False,"07 27, 2014",A89F3LQADZBS5,7106116521,Alexander D.,Okay,Three Stars,1406419200,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3631,5,True,"02 21, 2017",A1ZSB2Q144UTEY,B01HJHTH5U,Amazon Customer,I absolutely love this dress!! It's sexy and ...,I absolutely love this dress,1487635200,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,[B077ZLGMJ3],NaN,NaN,NaN,NaN
3632,5,True,"11 25, 2016",A2CCDV0J5VB6F2,B01HJHTH5U,Amazon Customer,I'm 5'6 175lbs. I'm on the tall side. I wear a...,I wear a large and ordered a large and it stil...,1480032000,2.0,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,[B077ZLGMJ3],NaN,NaN,NaN,NaN
3633,3,True,"11 10, 2016",A3O90PACS7B61K,B01HJHTH5U,Fabfifty,Too big in the chest area!,Three Stars,1478736000,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,[B077ZLGMJ3],NaN,NaN,NaN,NaN
3634,3,True,"11 10, 2016",A2HO94I89U3LNH,B01HJHF97K,Mgomez,"Too clear in the back, needs lining",Three Stars,1478736000,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Step 2: Loading Sentiment Analysis Model (Self-trained)

In [7]:
filename = 'vectorizer_full_amazon.pk'
filepath = amazon_model_folderpath.joinpath(filename)
vectorizer = pk.load(open(filepath, 'rb'))

filename = 'logreg_model_full_amazon.sav'
filepath = amazon_model_folderpath.joinpath(filename)
loaded_model = pk.load(open(filepath, 'rb'))

In [8]:
def logreg_model(TEST_REVIEW, negation_tag):
    test_vectors = vectorizer.transform([TEST_REVIEW])
    predME = loaded_model.predict(test_vectors)
    pred= list(predME)
    if pred==[1.0]:
        if negation_tag:
            prediction = "Negative"
        else:
            prediction = "Positive"
    else:
        if negation_tag:
            prediction = "Positive"
        else:
            prediction = "Negative"
    return prediction

In [9]:
print(logreg_model("This shirt is good", False))
print(logreg_model("This shirt is bad", False))
print(logreg_model("This shirt is not bad", False))

Positive
Negative
Negative


In [10]:
filename = 'vectorizer_full_amazon.pk'
filepath = amazon_model_folderpath.joinpath(filename)
vectorizer = pk.load(open(filepath, 'rb'))

filename = 'SVM_model_full_amazon.sav'
filepath = amazon_model_folderpath.joinpath(filename)
loaded_model = pk.load(open(filepath, 'rb'))

In [11]:
def SVM_model(TEST_REVIEW, negation_tag):
    test_vectors = vectorizer.transform([TEST_REVIEW])
    predME = loaded_model.predict(test_vectors)
    pred= list(predME)
    if pred==[1.0]:
        if negation_tag:
            prediction = "Negative"
        else:
            prediction = "Positive"
    else:
        if negation_tag:
            prediction = "Positive"
        else:
            prediction = "Negative"
    return prediction

In [12]:
print(SVM_model("This shirt is good", False))
print(SVM_model("This shirt is bad", False))
print(SVM_model("This shirt is not bad", False))

Positive
Negative
Positive


In [13]:
#Trained Spacy Model
def spacy_model(TEST_REVIEW, negation_tag):
    url_primary = 'https://sentiment.danielthx.repl.co/input/' + str(TEST_REVIEW)
    url_secondary = 'https://danieltanhx.pythonanywhere.com/?input=' + str(TEST_REVIEW)

    try:
        r = requests.get(url_primary)
        pred = json.loads(r.text)["Predicted sentiment"]

        if negation_tag:
            if pred == 'Positive':
                prediction = 'Negative'
            elif pred == 'Negative':
                prediction = 'Positive'
        else:
            prediction = pred

        return prediction

    except:
        r = requests.get(url_secondary)
        pred = json.loads(r.text)["Predicted sentiment"]

        if negation_tag:
            if pred == 'Positive':
                prediction = 'Negative'
            elif pred == 'Negative':
                prediction = 'Positive'
        else:
            prediction = pred

        return prediction

In [14]:
print(spacy_model("This shirt is good", False))
print(spacy_model("This shirt is bad", False))
print(spacy_model("This shirt is not bad", False))

Positive
Negative
Positive


# Step 3: Determine Nouns and Adjective Keywords For Different Aspects

In [15]:
# mystopwords = stopwords.words("english")
# WNlemma = nltk.WordNetLemmatizer()

# def pre_process(text):
#     text_pos = nltk.pos_tag(nltk.word_tokenize(str(text)))
#     tokens   = [ t for t in lemmaNVAR(text_pos) ]
#     tokens   = [ t for t in tokens if t not in mystopwords]
#     tokens   = [ t for t in tokens if len(t) >= 3 ]
#     return(tokens)

# def lemmaNVAR(wpos):
#     lemmas = []
#     for w, pos in wpos:
#         if pos == 'NNS':
#             lemmas.append(WNlemma.lemmatize(w.lower(), pos = pos[0].lower()))
#     return lemmas

# reviewText = review_with_meta["reviewText"].apply(pre_process)

In [16]:
# my_stopwords =  stopwords.words('english') + ['#']

# def preprocess_noun(doc):
#     doc = nlp(str(doc))
#     noun_list = [token for token in doc if token.pos_== "NOUN" and not token.text.isnumeric()]
#     noun_list = [noun.lemma_ for noun in noun_list if noun.text not in my_stopwords]

#     return noun_list

# def preprocess_adjective(doc):
#     doc = nlp(str(doc))
#     adjective_list = [token for token in doc if token.pos_ == "ADJ" and not token.text.isnumeric()]
#     adjective_list = [adjective.lemma_ for adjective in adjective_list if adjective.text not in my_stopwords]
    
#     return adjective_list

In [17]:
# noun_list = preprocess_noun(review_with_meta["reviewText"].iloc[1])
# adjective_list = preprocess_adjective(review_with_meta["reviewText"].iloc[1])

# print(noun_list)
# print(adjective_list)

In [18]:
# import time
# start_time = time.time()
# noun_list = review_with_meta["reviewText"].apply(preprocess_noun)
# end_time = time.time()
# print(f"Time taken {end_time-start_time}")

In [19]:
# import time
# start_time = time.time()
# adjective_list = review_with_meta["reviewText"].apply(preprocess_adjective)
# end_time = time.time()
# print(f"Time taken {end_time-start_time}")

In [20]:
# noun_toks = [toks for tokens in noun_list for toks in tokens]
# fdist = FreqDist(noun_toks)
# fdist.most_common(100)

In [21]:
# adjective_toks = [toks for tokens in adjective_list for toks in tokens]
# fdist = FreqDist(adjective_toks)
# fdist.most_common(100)

# Step 4: Aspect Detection Model

In [22]:
topic_list = ['size', 'comfort', 'appearance', 'quality', 'price', 'delivery']

In [23]:
# Aspect Types from common Amazon data
noun_keywords = { 'size'          : ['size', 'fit', 'length'],
                  'comfort'       : [],
                  'appearance'    : ['colour', 'picture', 'design', 'style', 'photo'],
                  'quality'       : ['quality', 'material', 'fabric', 'leather'],
                  'price'         : ['price', 'money'],
                  'delivery'      : ['time', 'day', 'seller', 'shipping', 'order']}
                     
for topic in topic_list:
    filename = topic+'.csv'
    filepath = wordnet_folderpath.joinpath(topic).joinpath(filename)
    
    with open(filepath, newline='\n') as f:
        reader = csv.reader(f)
        data = list(reader)
    nouns = noun_keywords[topic] + [pair[0] for pair in data if pair[1] == 'noun']
    noun_keywords[topic] = list(dict.fromkeys(nouns))
    print(noun_keywords[topic])

['size', 'fit', 'length', 'medium', 'bulkiness', 'enormousness', 'vastness', 'hang', '8vo', 'tininess', 'enormity', '4to', 'minuteness', 'narrow gauge', 'airiness', 'octavo', 'wingspan', 'voluminousness', 'slightness', 'diminutiveness', 'wingspread', 'gigantism', "arm's length", 'number', 'delicacy', 'stuntedness', 'capaciousness', 'giantism', 'spaciousness', 'circumference', 'skip distance', 'seating capacity', 'ampleness', 'littleness', 'greatness', 'grandness', 'spread', 'petite', 'wideness', 'stout', 'extra large', 'gauge', 'regular', 'immenseness', 'dovetail', 'perimeter', 'dwarfishness', 'bigness', 'broad gauge', 'quarto', 'runtiness', 'girth', 'tall', 'small', 'sizeableness', 'smallness', 'petiteness', 'leg', 'large', 'roominess', 'fitting', 'puniness', 'immensity', 'grain', 'commodiousness', 'standard gauge', 'joint', 'voluminosity', 'light time', 'fullness', 'outsize', 'weeness', 'eightvo', 'distance', 'largeness', 'massiveness', 'fitter', 'altitude']
['solace', 'alleviation',

In [24]:
# Aspect Types from common Amazon data
adjective_keywords = { 'size'          : ['small', 'large', 'little', 'big', 'fit', 'long', 'short', 'tight', 'loose', 'medium', 'tiny', 'huge'],
                       'comfort'       : ['comfortable', 'uncomfortable', 'soft', 'lightweight','comfy'],
                       'appearance'    : ['beautiful', 'stylish', 'flattering', 'gorgeous', 'lovely', 'sexy', 'adorable', 'cool'],
                       'quality'       : ['durable', 'sturdy', 'heavy', 'thick', 'new', 'old', 'hard'],
                       'price'         : ['cheap', 'expensive', 'honest', 'worth'],
                       'delivery'      : ['fast', 'quick']}

for topic in topic_list:
    filename = topic+'.csv'
    filepath = wordnet_folderpath.joinpath(topic).joinpath(filename)
    
    with open(filepath, newline='\n') as f:
        reader = csv.reader(f)
        data = list(reader)
    adjectives = adjective_keywords[topic] + [pair[0] for pair in data if pair[1] == 'adj']
    adjective_keywords[topic] = list(dict.fromkeys(adjectives))
    print(adjective_keywords[topic])

['small', 'large', 'little', 'big', 'fit', 'long', 'short', 'tight', 'loose', 'medium', 'tiny', 'huge', 'runty', 'voluminous', 'diminutive', 'average', 'commodious', 'medial', 'stunted', 'puny', 'sizeable', 'wee', 'immense', 'massive', 'outsize', 'great', 'dwarfish', 'ample', 'wide', 'vast', 'airy', 'altitudinal', 'intermediate', 'capacious', 'roomy', 'enormous', 'minute', 'slight', 'grand', 'full', 'petite', 'bulky', 'stout', 'spacious']
['comfortable', 'uncomfortable', 'soft', 'lightweight', 'comfy', 'cozy', 'cosy', 'snug', 'convenient']
['beautiful', 'stylish', 'flattering', 'gorgeous', 'lovely', 'sexy', 'adorable', 'cool', 'unsightly', 'showy', 'fair', 'mellow', 'glorious', 'grey', 'homely', 'sapphire', 'rosy', 'vinaceous', 'out-of-date', 'nice', 'florid', 'outdated', 'dark', 'cerise', 'greenish', 'ultramarine', 'crimson', 'stripy', 'splashy', 'lurid', 'wan', 'gray', 'garish', 'iron-grey', 'iron-gray', 'azure', 'rich', 'straw', 'chestnut', 'blotchy', 'sanguineous', 'superficial', '

In [25]:
def find_noun_head(token):
    #print(f"Finding Token: {token}")
    #print(f"Token Head: {token.head}")
    if token.head.pos_ == "NOUN" or token.head.pos_ == "NOUN" or token.head.dep_ == "ROOT":
        #print("Found")
        return token.head
    else:
        #print("Continue Search")
        return find_noun_head(token.head)
    
def find_negation_tag(adjective):
    #print(f"Finding Negatation for Adjective: {adjective}")
    #print(f"Token Head: {adjective.head}")
    if adjective.head.dep_ == "neg":
        return True
    elif adjective.head.pos_ == "AUX":
        for child in adjective.head.children:
            if child.dep_ =='neg':
                return True
    return False

def find_span_start(sent, token):
    #for tok in sent:
    #    print(f"{tok.i}: {tok} - {tok.pos_}")
    #print(f"Start {token.i}: {token} - {token.pos_}")  
    if token.i == sent.start or sent[token.i-1].pos_ == "PUNCT":
        #print(token.i)
        return token.i
    else:
        #print(token.i-1)
        return find_span_start(sent, sent[token.i-1])

def find_span_end(sent, token):
    #for tok in sent:
    #    print(f"{tok.i}: {tok} - {tok.pos_}")
    #print(f"End {token.i}: {token} - {token.pos_}")        
    if token.i+1 == sent.end or sent[token.i+1].pos_ == "PUNCT":
        #print(token.i+1)
        return token.i+1
    else:
        #print(token.i+1)
        return find_span_end(sent, sent[token.i+1])
    
def match_topics(noun_token, adjective_token, topic_list, noun_keywords, adjective_keywords):

    for topic in topic_list:
        if noun_token.lemma_ in noun_keywords[topic]:
            return topic
        elif adjective_token.lemma_ in adjective_keywords[topic]:
            return topic

    return None

In [27]:
#aggregate
def adsa_model(review, DEBUG = False, SENTI_MODEL = 'logreg'):

    #Model Setting
    SENTI_MODEL = SENTI_MODEL
    DEBUG = DEBUG

    #Model    
    doc = nlp(review)

    topic_prediction = {'size'          : None,
                        'comfort'       : None,
                        'appearance'    : None,
                        'quality'       : None,
                        'price'         : None, 
                        'delivery'      : None}
    
    topic_positive = {'size'          : 0.0,
                        'comfort'     : 0.0,
                        'appearance'  : 0.0,
                        'quality'     : 0.0,
                        'price'       : 0.0, 
                        'delivery'    : 0.0}
    
    topic_total = {'size'             : 0.0,
                        'comfort'     : 0.0,
                        'appearance'  : 0.0,
                        'quality'     : 0.0,
                        'price'       : 0.0, 
                        'delivery'    : 0.0}
    
    sent_count = 0
    descriptor_pair = []
    
    for sent in doc.sents:
        sent_count += 1

        adjectives = [tok for tok in sent if tok.pos_ == "ADJ"]
        pronouns = [tok for tok in sent if tok.pos_ == "PRON"]
        nouns = [tok for tok in sent if tok.pos_ == "NOUN"]
        negations = [tok for tok in sent if tok.dep_ == "neg"]
        
        if DEBUG:
            print(f"Sentence {sent_count}: {sent}")
            displacy.render(sent, style="dep")
            print(f"Adjectives: {adjectives}")
            print(f"Nouns: {nouns}")
            print(f"Pronouns: {pronouns}")
            print(f"Negations: {negations}")
            print("")

        for adjective in adjectives:
            isFound = False
            topic = None
            
            try:
                descriptor = ""
                for child in adjective.children:
                    if child.pos_ == "ADV":
                        descriptor += child.text + " "
                descriptor += adjective.text

                negation_tag = find_negation_tag(adjective)


                #Direct (i.e. The dress has a beautiful colour)
                noun = find_noun_head(adjective)
                if noun.pos_ == "NOUN" or noun.pos_ == "PRON":
                    trace = f"Direct Reference Detected for Adjective: {adjective}"
                    for chunk in sent.noun_chunks:
                        if chunk.root == noun and adjective not in chunk:
                            isFound = True
                            noun_subj = chunk
                        else:
                            isFound = True
                            noun_subj = noun             

                #Passive Voice (i.e. Colour of the dress was beautiful)
                elif adjective.head.pos_ == "AUX" or adjective.head.pos_ == "VERB":
                    trace = f"Indirect Reference Detected for Adjective: {adjective}"
                    for child in adjective.head.children:
                        if child.pos_ == "NOUN" or child.pos_ == "PRON":
                            noun = child                      
                            for chunk in sent.noun_chunks:
                                if chunk.root == child and adjective not in chunk:
                                    isFound = True
                                    noun_subj = chunk
                                else:
                                    isFound = True
                                    noun_subj = noun
                                    
                #Guessing when improper grammer is used (i.e. Colour of the dress was beautiful vs also dress colour beautiful)
                else:
                    trace = f"Finding noun subject for Adjective: {adjective}\n"
                    start = find_span_start(sent, adjective)
                    end = find_span_end(sent, adjective)
                    extract = sent[start:end]
                    trace += f"Extract: {extract}\n"

                    for token in extract:
                        if token.pos_ == "NOUN" and token.dep_ == "nsubj":
                            isFound = True
                            noun_subj = token
                        if token.dep_ == "neg":
                            negation_tag = True
            except:
                isFound = False

            if isFound:
                topic = match_topics(noun, adjective, topic_list, noun_keywords, adjective_keywords)
            else:
                trace += f"Matching adjective without noun for Adjective: {adjective}"
                noun_subj = None
                topic = match_topics(nlp("I")[0], adjective, topic_list, noun_keywords, adjective_keywords)

            if topic != None:
                if noun_subj == None:                    
                    if SENTI_MODEL == 'logreg':
                        prediction = logreg_model(str(descriptor) + " " + str(topic), negation_tag)
                    elif SENTI_MODEL == 'svm':
                        prediction = SVM_model(str(descriptor) + " " + str(topic), negation_tag)
                    elif SENTI_MODEL == 'spacy':
                        prediction = spacy_model(str(descriptor) + " " + str(topic), negation_tag)
                else:
                    if SENTI_MODEL == 'logreg':
                        prediction = logreg_model(str(descriptor) + " " + str(noun_subj), negation_tag)
                    elif SENTI_MODEL == 'svm':
                        prediction = SVM_model(str(descriptor) + " " + str(noun_subj), negation_tag)
                    elif SENTI_MODEL == 'spacy':
                        prediction = spacy_model(str(descriptor) + " " + str(noun_subj), negation_tag)
                    

                descriptor_pair.append((descriptor, noun_subj, negation_tag, topic, prediction))

            if DEBUG:
                print(f"Trace: {trace}")
                if topic != None:
                    print(f"Descriptor_pair: {(descriptor, noun_subj, negation_tag, topic, prediction)}")
                print("")
    if DEBUG:
        print(f"All descriptor_pair: {descriptor_pair}")
        print("")
    
    for pair in descriptor_pair:
        topic_total[pair[3]] += 1
        if pair[4] == "Positive":
            topic_positive[pair[3]] += 1
            
    for topic in topic_list:
        if topic_total[topic] != 0 and (topic_positive[topic] / topic_total[topic]) >= 0.5:
            topic_prediction[topic] = "Positive"
        elif topic_total[topic] != 0 and (topic_positive[topic] / topic_total[topic]) < 0.5:
            topic_prediction[topic] = "Negative"
        else:
            topic_prediction[topic] = None
                
                
    print(f"Review: {review}")
    print(f"Topic_prediction: {topic_prediction}")
    print("---------------------------------------------------------------")
    print("")
    
    return topic_prediction

# Step 5a: Evaluation Functions

In [28]:
def process_evaluation(spans):
    
    topic_list = ['size', 'comfort', 'appearance', 'quality', 'price', 'delivery']    
    topic_prediction = {'size'          : None,
                        'comfort'       : None,
                        'appearance'    : None,
                        'quality'       : None,
                        'price'         : None, 
                        'delivery'      : None}
    
    if not isinstance(spans, float):
        for span in spans:
            topic_prediction[span['label'].lower()] = "Found"

    return topic_prediction

In [29]:
def accuracy(series):
    topic_list = ['size', 'comfort', 'appearance', 'quality', 'price', 'delivery']    
    
    correct = 0
    for topic in topic_list:
        if series['topics_target'][topic] != None and series['topics_predicted'][topic] != None:
            correct += 1
        elif series['topics_target'][topic] == None and series['topics_predicted'][topic] == None:
            correct += 1
    
    accuracy = correct / len(topic_list)
    
    return accuracy

In [30]:
def to_list(dict):
    topic_list = ['size', 'comfort', 'appearance', 'quality', 'price', 'delivery']
    
    list = []
    
    for topic in topic_list:
        if dict[topic] != None:
            list.append(1)
        else:
            list.append(0)
    
    return list

# Step 5b: Apply Model to Evaluation Dataset (Amazon)

In [31]:
filename = 'amazon_adsa_ner.jsonl'
filepath = data_folderpath.joinpath(filename)
amazon_test_data = pd.read_json(filepath, lines=True)

In [32]:
amazon_test_data['topics_target'] = amazon_test_data['spans'].apply(process_evaluation)

In [33]:
amazon_test_data['topics_predicted'] = amazon_test_data['text'].apply(adsa_model, SENTI_MODEL = 'spacy')

Review: nut Malo es very cheap
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': 'Negative', 'delivery': None}
---------------------------------------------------------------

Review: It seems the sizing of this is way off.  I read the reviews and ordered a size larger than I'd normally wear and it was still WAY TOO SMALL!
Topic_prediction: {'size': 'Positive', 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: they are too small they did not even fit on my ear i would not recommend this product they are so tiny its a joke
Topic_prediction: {'size': 'Negative', 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: I thought I got the right size but apparently not. WAY too small. Order up a size (I'm a cross dresser, and this look

Review: These are great insoles!  I went to the podiatrist and he immediately said he was glad to see I was wearing powersteps in my shoes.  My whole leg feels better when I wear these.  The have good arch support.
Topic_prediction: {'size': 'Positive', 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: Overall this is a very cute shirt, I followed the recommend sizing from other reviewers and the shirt was a bit too big. It is not a slimming shirt, it would be great if you are pregnant as it has a lot of extra room. I tried using a belt and it didn't help at all. It doesn't hang like it shows in the picture. I do love the color and the chest and sleeves did fit well. It also only hangs to about your waste so even making it work with leggings wasn't happening for me. I would either try to size to your current size or even size down. It hangs extremely loosely past your chest. It

Review: Amazing, photos to come!
Topic_prediction: {'size': None, 'comfort': None, 'appearance': 'Positive', 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: Cheep very cheep
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: I think this ring is GORGEOUS but some of the side diamonds are put in sideways.
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: Too thine . But nice
Topic_prediction: {'size': None, 'comfort': None, 'appearance': 'Positive', 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: Perfect. Exactly what I wanted. Great price
Topic_prediction: {'

Review: I got the style two which was beautiful fabric, loved it! I was so excited that I found a style where everything tied so it was more adjustable.  The top fit perfectly......now for the but.......unfortunately for me and for the low rating, the coverage for the gluteus was showing too maxims, in other words didn't cover enough for my for my taste.
Topic_prediction: {'size': 'Positive', 'comfort': None, 'appearance': 'Positive', 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: I got these recently to wear for sporting activities. I have to say they are probably the best and most comfortable I've ever own. Very high quality and the size is perfect. I highly recommend this product!
Topic_prediction: {'size': None, 'comfort': 'Positive', 'appearance': None, 'quality': 'Positive', 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: The material is quite thin

Review: These work incredibly well.  Previous retainers would stretch out where they attached to the eye glasses.  These are very secure.
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': 'Positive', 'delivery': None}
---------------------------------------------------------------

Review: This were nice.  A thin material but great for sleeping and working out.
Topic_prediction: {'size': 'Positive', 'comfort': None, 'appearance': 'Positive', 'quality': 'Negative', 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: I'd give it fewer stars... I'm a fairly petite person. I ordered a Medium, hoping for a really loose feel. It was so tight, the neck felt uncomfortably similar to a choker.
Topic_prediction: {'size': 'Negative', 'comfort': None, 'appearance': 'Negative', 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: F

...

Review: It u have a gut no tummy control
it does lift your butt a little but it only works in jeans not tights or sweat pants
Topic_prediction: {'size': 'Positive', 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: fits grate
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: The material is cheap and thin. I am 5'3" 120 lbs and the small is much too big and baggie. It's unfortunate because the dress itself is quite  pretty.
Topic_prediction: {'size': 'Negative', 'comfort': None, 'appearance': None, 'quality': 'Negative', 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: Very trendy, well made, fits nice.  Avoid water on the gold.
Topic_prediction: {'size': None, 'comfort': None, 'ap

Review: Love this shirt. Wear it all the time. Fits loose and comfortable. Love the material.
Topic_prediction: {'size': 'Negative', 'comfort': 'Positive', 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: not at all as advertised its T-shirt material and thrown together and way to small!!
Topic_prediction: {'size': 'Negative', 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: My wife gave her other Columbia rain jacket away to her sister in law who lives overseas and cannot find any at a similar price <$50.  She is pleased with this one as much as the first one.
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': 'Positive', 'delivery': 'Positive'}
---------------------------------------------------------------

Review: My daughter ordered a 

Review: To big but I hues I didn't covert size right
Topic_prediction: {'size': 'Positive', 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: It was pretty! Just not for me
Topic_prediction: {'size': None, 'comfort': None, 'appearance': 'Positive', 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: grandaughter love it, she a kitty lover
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: This case came quickly and the case itself looks fine & fits onto the iphone, but is much bulkier than it looks in the pictures. I have an iphone 6, not a 6s so maybe that is the problem (though it says it's made for both). The buttons like the sound on/off and the headphone port are 

Review: Super soft.  Fits great.  Very happy with this purchase.
Topic_prediction: {'size': 'Positive', 'comfort': 'Positive', 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: it was very nice.  my grandson picked it out and was very happy with it.  he said it was very nice.
Topic_prediction: {'size': None, 'comfort': None, 'appearance': 'Positive', 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: Awkward and uncomfortable. I should have returned. Bite the bullet and purchase maternity jeans.
Topic_prediction: {'size': None, 'comfort': 'Negative', 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: I had to buy two of these rings because the 1st one a stone fell out. Now the 2nd one same side, same stone came out. i won't be buying

Review: No what I was expecting. I know that it is a good price, but seems like plastic
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': 'Positive', 'delivery': None}
---------------------------------------------------------------

Review: If I would of had the knowledge that ot was going to be a walmart shirt we would of ordered a medium size. Sad thst 3rd party sealers that are not connected with walmart use their products and charge 3 times more for them. Beware of this happening to you.
Topic_prediction: {'size': 'Positive', 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: A gift for my Grand daughter.
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: Nephew loved this
Topic_prediction:

In [34]:
amazon_test_data['accuracy'] = amazon_test_data[['topics_target', 'topics_predicted']].apply(accuracy, axis=1)

In [35]:
amazon_test_data['target_list'] = amazon_test_data['topics_target'].apply(to_list)
amazon_test_data['predicted_list'] = amazon_test_data['topics_predicted'].apply(to_list)

In [36]:
amazon_target_list = []
amazon_predicted_list = []

for index in range(len(amazon_test_data)):
    amazon_target_list.append(amazon_test_data.iloc[index]['target_list'])
    amazon_predicted_list.append(amazon_test_data.iloc[index]['predicted_list'])

In [37]:
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import classification_report

print(multilabel_confusion_matrix(amazon_target_list, amazon_predicted_list))
print(classification_report(amazon_target_list, amazon_predicted_list))

[[[208  61]
  [ 39 152]]

 [[385   3]
  [ 24  48]]

 [[254  36]
  [ 54 116]]

 [[317  23]
  [ 71  49]]

 [[406  20]
  [ 16  18]]

 [[416  10]
  [ 19  15]]]
              precision    recall  f1-score   support

           0       0.71      0.80      0.75       191
           1       0.94      0.67      0.78        72
           2       0.76      0.68      0.72       170
           3       0.68      0.41      0.51       120
           4       0.47      0.53      0.50        34
           5       0.60      0.44      0.51        34

   micro avg       0.72      0.64      0.68       621
   macro avg       0.70      0.59      0.63       621
weighted avg       0.73      0.64      0.67       621
 samples avg       0.52      0.50      0.49       621



c:\users\onnwe\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\onnwe\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [38]:
amazon_test_data[amazon_test_data['accuracy'] != 1.0]

,text,_input_hash,_task_hash,tokens,spans,_is_binary,_session_id,_view_id,answer,_timestamp,topics_target,topics_predicted,accuracy,target_list,predicted_list
5,Not well made- you get what you pay for...,-18663828,913049618,"[{'text': 'Not', 'start': 0, 'end': 3, 'id': 0...","[{'start': 0, 'end': 14, 'token_start': 0, 'to...",False,NaN,ner_manual,accept,1636195565,"{'size': None, 'comfort': None, 'appearance': ...","{'size': None, 'comfort': None, 'appearance': ...",0.833333,"[0, 0, 0, 1, 0, 0]","[0, 0, 0, 0, 0, 0]"
7,I ordered a size 4 which should have fit me pe...,1718264975,-1594482082,"[{'text': 'I', 'start': 0, 'end': 1, 'id': 0, ...","[{'start': 70, 'end': 87, 'token_start': 15, '...",False,NaN,ner_manual,accept,1636195737,"{'size': 'Found', 'comfort': None, 'appearance...","{'size': 'Negative', 'comfort': None, 'appeara...",0.666667,"[1, 0, 0, 1, 0, 1]","[1, 0, 1, 0, 0, 1]"
11,"The dress is very pretty, but it runs very sma...",447260364,258601340,"[{'text': 'The', 'start': 0, 'end': 3, 'id': 0...","[{'start': 13, 'end': 24, 'token_start': 3, 't...",False,NaN,ner_manual,accept,1636195809,"{'size': 'Found', 'comfort': None, 'appearance...","{'size': 'Negative', 'comfort': 'Positive', 'a...",0.666667,"[1, 0, 1, 0, 0, 0]","[1, 1, 1, 1, 0, 0]"
12,Its a little thinner than I anticipated but I'...,-1651956451,-1775836404,"[{'text': 'Its', 'start': 0, 'end': 3, 'id': 0...","[{'start': 6, 'end': 20, 'token_start': 2, 'to...",False,NaN,ner_manual,accept,1636195837,"{'size': None, 'comfort': None, 'appearance': ...","{'size': 'Positive', 'comfort': None, 'appeara...",0.500000,"[0, 0, 0, 1, 1, 0]","[1, 0, 0, 0, 0, 0]"
17,When I got this I was very pleased but had it ...,1534071032,-229267721,"[{'text': 'When', 'start': 0, 'end': 4, 'id': ...","[{'start': 174, 'end': 201, 'token_start': 42,...",False,NaN,ner_manual,accept,1636195920,"{'size': None, 'comfort': None, 'appearance': ...","{'size': None, 'comfort': None, 'appearance': ...",0.833333,"[0, 0, 0, 1, 0, 0]","[0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,The dress had a very strong smell of factory. ...,-533040091,-1844490635,"[{'text': 'The', 'start': 0, 'end': 3, 'id': 0...","[{'start': 16, 'end': 44, 'token_start': 4, 't...",False,NaN,ner_manual,accept,1636257683,"{'size': None, 'comfort': None, 'appearance': ...","{'size': None, 'comfort': None, 'appearance': ...",0.833333,"[0, 0, 1, 1, 0, 0]","[0, 0, 1, 0, 0, 0]"
456,Nice for the money. This is for Xmas.,-67636620,1347205755,"[{'text': 'Nice', 'start': 0, 'end': 4, 'id': ...","[{'start': 0, 'end': 18, 'token_start': 0, 'to...",False,NaN,ner_manual,accept,1636257687,"{'size': None, 'comfort': None, 'appearance': ...","{'size': None, 'comfort': None, 'appearance': ...",0.666667,"[0, 0, 0, 0, 1, 0]","[0, 0, 1, 0, 0, 0]"
457,I am so THRILLED about this dress! The fabric...,1897674455,209258134,"[{'text': 'I', 'start': 0, 'end': 1, 'id': 0, ...","[{'start': 40, 'end': 72, 'token_start': 10, '...",False,NaN,ner_manual,accept,1636257706,"{'size': 'Found', 'comfort': None, 'appearance...","{'size': 'Negative', 'comfort': None, 'appeara...",0.833333,"[1, 0, 1, 1, 0, 0]","[1, 0, 1, 1, 1, 0]"
458,Perfect for prom. Everyone loved my dress and ...,-1550624492,181493175,"[{'text': 'Perfect', 'start': 0, 'end': 7, 'id...","[{'start': 46, 'end': 62, 'text': 'it fit perf...",False,NaN,ner_manual,accept,1636257712,"{'size': 'Found', 'comfort': None, 'appearance...","{'size': None, 'comfort': None, 'appearance': ...",0.666667,"[1, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0]"


In [39]:
index = 11

print(amazon_test_data.iloc[index]['text'])
print(amazon_test_data.iloc[index]['topics_target'])
print(amazon_test_data.iloc[index]['topics_predicted'])
print(amazon_test_data.iloc[index]['accuracy'])

The dress is very pretty, but it runs very small in the waist and very large in the bust. If you're not an extreme hour-glass, be prepared to alter this before wearing it. I'm short, so I plan to take some fabric either from the hem or the oversized bust to redistribute as needed, personally.

Not as comfortable as some of the other items I've gotten from SheIn before - the fabric is a little scratchier than anticipated - but it does flow nicely when you move around.
{'size': 'Found', 'comfort': None, 'appearance': 'Found', 'quality': None, 'price': None, 'delivery': None}
{'size': 'Negative', 'comfort': 'Positive', 'appearance': 'Positive', 'quality': 'Positive', 'price': None, 'delivery': None}
0.6666666666666666


In [40]:
import numpy as np

np.average(amazon_test_data['accuracy'])

0.8637681159420291

# Step 6: Apply Model to Evaluation Dataset (Shopee)

In [41]:
filename = 'shopee_adsa_ner.jsonl'
filepath = data_folderpath.joinpath(filename)
shopee_test_data = pd.read_json(filepath, lines=True)

In [42]:
shopee_test_data['topics_target'] = shopee_test_data['spans'].apply(process_evaluation)

In [43]:
shopee_test_data['topics_predicted'] = shopee_test_data['text'].apply(adsa_model, SENTI_MODEL = 'spacy')

Review: Very nice blouses. Received before CNY.Nicely fitting on me. Thanks seller.I will buy again.
Topic_prediction: {'size': None, 'comfort': None, 'appearance': 'Positive', 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: I like the material of this shirt — very breathable and doesn’t show in an obvious manner when I have sweat a lot in it. Will consider repurchasing 👍🏼
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': 'Positive', 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: Perfect fit and nice material. Worth to consider if you look for a work out tank too. 👍🏻😊
Topic_prediction: {'size': 'Positive', 'comfort': None, 'appearance': 'Positive', 'quality': 'Positive', 'price': 'Positive', 'delivery': None}
---------------------------------------------------------------

Review: Order based on size chart, not suitable. Mu

Review: Item came as described but the size for the pants is too tight even though the shirt is big enough:(
Topic_prediction: {'size': 'Positive', 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: the t shirt is comfortable worth the price
Topic_prediction: {'size': None, 'comfort': 'Positive', 'appearance': None, 'quality': None, 'price': 'Positive', 'delivery': None}
---------------------------------------------------------------

Review: Haven't tested the quality, but seems good to me. Will buy again
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: Perfect i love the length of the sleeves and the slits at the end of the sleeves 
Topic_prediction: {'size': 'Positive', 'comfort': None, 'appearance': None, 'quality': None, 'price': No

Review: Fast delivery! Nice mint colour! I will order from seller again if there any designs that I like.
Topic_prediction: {'size': None, 'comfort': None, 'appearance': 'Positive', 'quality': None, 'price': None, 'delivery': 'Positive'}
---------------------------------------------------------------

Review: Good fit! I didn't want it to look oversized (altho the sleeves will look oversized). I usually wear uk8/10. The quality feels good too
Topic_prediction: {'size': 'Positive', 'comfort': None, 'appearance': None, 'quality': 'Positive', 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: Highly recommended. Soft And comfy. Keep me from cold. Not too translucent. Relatively thick . Makes me want to buy more!!! Colours
Topic_prediction: {'size': None, 'comfort': 'Positive', 'appearance': None, 'quality': 'Positive', 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: Nice fittin

Review: Product material was good, received quite fast.. Will purchase again..
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': 'Positive', 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: Items arrived as per pictures. Goods arrived within 2wks of order. The quality of the purple one is thicker but looks firmer
Topic_prediction: {'size': None, 'comfort': None, 'appearance': 'Positive', 'quality': 'Positive', 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: Item took awhile to arrive and there were a few loose threads but they came in pretty good condition. Size up if you want a looser pyjamas. Otherwise, material was so much better than I expected. 
Topic_prediction: {'size': 'Positive', 'comfort': None, 'appearance': None, 'quality': 'Positive', 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: nice
Topic_prediction: {'size': None, 'comfort': None, 'appearance': 'Positive', 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: Received it in 4 days!! 2XL was big for me but I can alter it. The quality of the material is nice! Overall, it was a good buy! 
Topic_prediction: {'size': 'Positive', 'comfort': None, 'appearance': 'Positive', 'quality': None, 'price': 'Positive', 'delivery': None}
---------------------------------------------------------------

Review: Good quality n good response from seller.
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': 'Positive', 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: It fits me perfectly. I live the material. Would definitely buy more👍
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
-----------------

...

Review: It’s too short for my liking. It’s more of a pad-less bra than a crop top :( can’t pair this with anything without showing too much skin. The material and quality is great tho. 
Topic_prediction: {'size': 'Positive', 'comfort': None, 'appearance': None, 'quality': 'Negative', 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: Looks good. Hopefully comfortable.
Topic_prediction: {'size': None, 'comfort': 'Positive', 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: Slightly translucent. Dots are quite big. But generally ok and quite fitting.
Topic_prediction: {'size': 'Positive', 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: Hopefully the straps can last.. else wear as a tube.. no complaints for the price paid
Topic_predi

Review: Cheap and use on tight brassiere. Good deal during flash sale cheaper than outside
Topic_prediction: {'size': 'Negative', 'comfort': None, 'appearance': None, 'quality': None, 'price': 'Negative', 'delivery': None}
---------------------------------------------------------------

Review: Quality is good and fits me well. I will order again! Delivery took a long time though. Thanks!
Topic_prediction: {'size': 'Positive', 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: Looks as per picture. Im size 16 fits quite snug but doesnt really give much support on the tummy area. Hopeful that the tighs area wont roll up throughout wears. 
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: Received with thanks. Bought 1 to try since wfh can 

Review: My repeat order. Soft and thin material pants. Value for money.
Topic_prediction: {'size': None, 'comfort': 'Positive', 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: Very soft material! Delivery took about 9 days. 
Topic_prediction: {'size': None, 'comfort': 'Positive', 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: Look smaller then I expected
Topic_prediction: {'size': 'Negative', 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review:  Good price not bad and fast delivery, with buy second later 👍👍

Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': 'Positive', 'delivery': 'Positive'}
--------------------------------------------------------

Review: Lovelovelove!! So pretty and good material!
Topic_prediction: {'size': None, 'comfort': None, 'appearance': 'Positive', 'quality': 'Positive', 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: quick delivery and items looks like its in good condition. thanks seller!
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': 'Positive'}
---------------------------------------------------------------

Review: Received in good condition. Quality and material of clothes is ok
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': 'Positive', 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: Have received the korea 13 colors spring summer loose short sleeve tops casual plain t-shirt. Recommended product 👍👍. Well packaged & smooth delivery. Recommended Seller: Xiaomaojia.sg -- Women Fashion. Th

Review: Nice bra. But not recommended for small boobs ladies. The cups are abit stiff. Otherwise, the price is good. 
Topic_prediction: {'size': 'Negative', 'comfort': None, 'appearance': 'Positive', 'quality': None, 'price': 'Positive', 'delivery': None}
---------------------------------------------------------------

Review: delivery was done super fast. received it within a week. love the top so much as well. thank you❤️😭
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: 😘😘😘😘😘😘😘😘Thank you, the goods have arrived and the delivery is very fast, thank you very much
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': 'Positive'}
---------------------------------------------------------------

Review: bought 3xl but its still very small, the white one is see through and very thin, the t

In [44]:
shopee_test_data['accuracy'] = shopee_test_data[['topics_target', 'topics_predicted']].apply(accuracy, axis=1)

In [45]:
shopee_test_data['target_list'] = shopee_test_data['topics_target'].apply(to_list)
shopee_test_data['predicted_list'] = shopee_test_data['topics_predicted'].apply(to_list)

In [46]:
shopee_target_list = []
shopee_predicted_list = []

for index in range(len(shopee_test_data)):
    shopee_target_list.append(shopee_test_data.iloc[index]['target_list'])
    shopee_predicted_list.append(shopee_test_data.iloc[index]['predicted_list'])

In [47]:
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import classification_report

print(multilabel_confusion_matrix(shopee_target_list, shopee_predicted_list))
print(classification_report(shopee_target_list, shopee_predicted_list))

[[[1077  153]
  [ 138  237]]

 [[1205   75]
  [  95  230]]

 [[1220  200]
  [  55  130]]

 [[ 982   98]
  [ 223  302]]

 [[1319  110]
  [  64  112]]

 [[ 848   74]
  [ 371  312]]]
              precision    recall  f1-score   support

           0       0.61      0.63      0.62       375
           1       0.75      0.71      0.73       325
           2       0.39      0.70      0.50       185
           3       0.76      0.58      0.65       525
           4       0.50      0.64      0.56       176
           5       0.81      0.46      0.58       683

   micro avg       0.65      0.58      0.62      2269
   macro avg       0.64      0.62      0.61      2269
weighted avg       0.70      0.58      0.62      2269
 samples avg       0.50      0.48      0.47      2269



c:\users\onnwe\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\onnwe\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [48]:
shopee_test_data[shopee_test_data['accuracy'] != 1.0]

,text,_input_hash,_task_hash,_is_binary,tokens,_session_id,_view_id,spans,answer,_timestamp,meta,topics_target,topics_predicted,accuracy,target_list,predicted_list
0,Very nice blouses. Received before CNY.Nicely ...,-1875761178,1154029328,False,"[{'text': 'Very', 'start': 0, 'end': 4, 'id': ...",NaN,ner_manual,"[{'start': 35, 'end': 59, 'token_start': 6, 't...",accept,1635676618,NaN,"{'size': 'Found', 'comfort': None, 'appearance...","{'size': None, 'comfort': None, 'appearance': ...",0.666667,"[1, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 0]"
2,Perfect fit and nice material. Worth to consid...,1572550529,-1577654101,False,"[{'text': 'Perfect', 'start': 0, 'end': 7, 'id...",NaN,ner_manual,"[{'start': 0, 'end': 11, 'token_start': 0, 'to...",accept,1635676653,NaN,"{'size': 'Found', 'comfort': None, 'appearance...","{'size': 'Positive', 'comfort': None, 'appeara...",0.666667,"[1, 0, 0, 1, 0, 0]","[1, 0, 1, 1, 1, 0]"
3,"Order based on size chart, not suitable. Must ...",-2021880665,411181325,False,"[{'text': 'Order', 'start': 0, 'end': 5, 'id':...",NaN,ner_manual,"[{'start': 41, 'end': 63, 'token_start': 9, 't...",accept,1635676665,NaN,"{'size': 'Found', 'comfort': None, 'appearance...","{'size': 'Positive', 'comfort': None, 'appeara...",0.833333,"[1, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 1]"
5,My lovely OREN T shirt.Comfort and cooling in ...,2062330765,359889990,False,"[{'text': 'My', 'start': 0, 'end': 2, 'id': 0,...",NaN,ner_manual,"[{'start': 23, 'end': 63, 'token_start': 6, 't...",accept,1635676706,NaN,"{'size': None, 'comfort': 'Found', 'appearance...","{'size': None, 'comfort': None, 'appearance': ...",0.500000,"[0, 1, 0, 0, 0, 1]","[0, 0, 1, 1, 0, 1]"
6,Quality is so gd I love it so so much thanku w...,1874603334,-887419603,False,"[{'text': 'Quality', 'start': 0, 'end': 7, 'id...",NaN,ner_manual,"[{'start': 0, 'end': 16, 'token_start': 0, 'to...",accept,1635676714,NaN,"{'size': None, 'comfort': None, 'appearance': ...","{'size': None, 'comfort': None, 'appearance': ...",0.833333,"[0, 0, 0, 1, 0, 0]","[0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1600,Received new loots today. Bought thermal pants...,762948577,-441648350,False,"[{'text': 'Received', 'start': 0, 'end': 8, 'i...",NaN,ner_manual,"[{'start': 122, 'end': 132, 'token_start': 23,...",accept,1636021383,NaN,"{'size': None, 'comfort': 'Found', 'appearance...","{'size': None, 'comfort': None, 'appearance': ...",0.666667,"[0, 1, 0, 1, 0, 0]","[0, 0, 0, 1, 0, 1]"
1601,Order received very fast!,-1590787560,633160436,False,"[{'text': 'Order', 'start': 0, 'end': 5, 'id':...",NaN,ner_manual,"[{'token_start': 0, 'token_end': 3, 'start': 0...",accept,1636021385,NaN,"{'size': None, 'comfort': None, 'appearance': ...","{'size': None, 'comfort': None, 'appearance': ...",0.833333,"[0, 0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0]"
1602,I loved the thin layer of the clothing n its q...,-522064259,-668210037,False,"[{'text': 'I', 'start': 0, 'end': 1, 'id': 0, ...",NaN,ner_manual,"[{'start': 0, 'end': 38, 'text': 'I loved the ...",accept,1636021397,NaN,"{'size': None, 'comfort': 'Found', 'appearance...","{'size': None, 'comfort': None, 'appearance': ...",0.666667,"[0, 1, 0, 1, 0, 0]","[0, 0, 0, 0, 0, 0]"
1603,"Quality isn’t that good, quite happy, I don’t ...",-1681328800,-1176781951,False,"[{'text': 'Quality', 'start': 0, 'end': 7, 'id...",NaN,ner_manual,"[{'start': 0, 'end': 23, 'text': 'Quality isn’...",accept,1636021410,NaN,"{'size': None, 'comfort': 'Found', 'appearance...","{'size': None, 'comfort': None, 'appearance': ...",0.833333,"[0, 1, 0, 1, 0, 0]","[0, 0, 0, 1, 0, 0]"


In [49]:
index = 5

print(shopee_test_data.iloc[index]['text'])
#print(shopee_test_data.iloc[index]['topics_target'])
#print(shopee_test_data.iloc[index]['topics_predicted'])
#print(shopee_test_data.iloc[index]['accuracy'])

My lovely OREN T shirt.Comfort and cooling in Singapore weather.
This is my 6th Oren T shirt (5 for working ) and this new one for exercise.
Thank you seller for the fast delivery.
Cheers


In [50]:
np.average(shopee_test_data['accuracy'])

0.8280373831775701

# Step 7: Apply Model to Selected Brands

In [51]:
brand_name = "adidas"

print(meta[meta["brand"]==brand_name])

                                                   title   brand  \
1512     adidas Women's ClimaCool Superlite No Show Sock  adidas   
5812    adidas Women's Response Short Sleeve Stretch Tee  adidas   
6126       adidas Women's Essentials 3-Stripe Knit Short  adidas   
6132              adidas Women's Essentials 3-Stripe Tee  adidas   
6384        ADIDAS Calissage black/metallic silver/white  adidas   
8934                                    adidas ZX 800 DB  adidas   
9882                    Adidas Attaak II Wrestling Shoes  adidas   
10392            adidas Women's Techfit Short-Sleeve Top  adidas   
11609  adidas Women's Ultimate Workout Short-Sleeve V...  adidas   
13699               Adidas Adi Firebird Tracktop #X41202  adidas   
57804                       adidas Boys' Triple Up Short  adidas   
69443  adidas Outdoor Kids Baby Girl's Daroga Leather...  adidas   
70852  adidas Women's Climacool Aeroknit Long Sleeve ...  adidas   
83933  adidas Manchester United FC Official 2015

In [52]:
reduced_review_with_meta = review_with_meta[review_with_meta["brand"]==brand_name]

In [53]:
reduced_review_with_meta

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,...,imageURL,imageURLHighRes,description,price,also_view,also_buy,fit,details,similar_item,tech1
3411,5,True,"05 17, 2016",AOU7T6BK2IQ1J,B00N9KLTSK,Byoung K.,Good,Five Stars,1463443200,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,[Children will be excited to get active in fas...,$9.95 - $24.99,"[B07D6MJRRL, B015SSQCO6, B01HO66CPE, B00CHNS6Y...","[B07D6MJRRL, B00U9TWIJS, B07CS9VTPB, B00CHNS6Y...","class=""a-normal a-align-center a-spacing-smal...","\n <div class=""content"">\n\n\n\n\n\n\n<ul...",NaN,NaN
3412,5,True,"02 9, 2016",A2V6HAVTGVWA8S,B00N9KLTSK,Kara M Lauve,Perfect fit,Five Stars,1454976000,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,[Children will be excited to get active in fas...,$9.95 - $24.99,"[B07D6MJRRL, B015SSQCO6, B01HO66CPE, B00CHNS6Y...","[B07D6MJRRL, B00U9TWIJS, B07CS9VTPB, B00CHNS6Y...","class=""a-normal a-align-center a-spacing-smal...","\n <div class=""content"">\n\n\n\n\n\n\n<ul...",NaN,NaN
3413,5,True,"02 4, 2016",A3GD1CS79C1JBN,B00N9KLTSK,sugarnay,works great. My son likes them.,Five Stars,1454544000,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,[Children will be excited to get active in fas...,$9.95 - $24.99,"[B07D6MJRRL, B015SSQCO6, B01HO66CPE, B00CHNS6Y...","[B07D6MJRRL, B00U9TWIJS, B07CS9VTPB, B00CHNS6Y...","class=""a-normal a-align-center a-spacing-smal...","\n <div class=""content"">\n\n\n\n\n\n\n<ul...",NaN,NaN
3414,5,True,"01 24, 2016",A2GCYCPA9VOEFR,B00N9KLTSK,Dustin P. Ward,they run a little large. No worries just roll ...,Sweet looking and very comfortable.,1453593600,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,[Children will be excited to get active in fas...,$9.95 - $24.99,"[B07D6MJRRL, B015SSQCO6, B01HO66CPE, B00CHNS6Y...","[B07D6MJRRL, B00U9TWIJS, B07CS9VTPB, B00CHNS6Y...","class=""a-normal a-align-center a-spacing-smal...","\n <div class=""content"">\n\n\n\n\n\n\n<ul...",NaN,NaN
3415,5,True,"12 27, 2015",A2ISCKJ7OLT1QO,B00N9KLTSK,Amazon Customer,My son loves these!,Five Stars,1451174400,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,[Children will be excited to get active in fas...,$9.95 - $24.99,"[B07D6MJRRL, B015SSQCO6, B01HO66CPE, B00CHNS6Y...","[B07D6MJRRL, B00U9TWIJS, B07CS9VTPB, B00CHNS6Y...","class=""a-normal a-align-center a-spacing-smal...","\n <div class=""content"">\n\n\n\n\n\n\n<ul...",NaN,NaN
3416,5,True,"11 30, 2015",AU1BSTL6EUKP,B00N9KLTSK,Jenna,My son loves basketball shorts. He would wear ...,Great shorts,1448841600,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,[Children will be excited to get active in fas...,$9.95 - $24.99,"[B07D6MJRRL, B015SSQCO6, B01HO66CPE, B00CHNS6Y...","[B07D6MJRRL, B00U9TWIJS, B07CS9VTPB, B00CHNS6Y...","class=""a-normal a-align-center a-spacing-smal...","\n <div class=""content"">\n\n\n\n\n\n\n<ul...",NaN,NaN
3417,5,True,"10 31, 2015",A38C8XNIETJLEU,B00N9KLTSK,Pen Name,"My son love it, the fabric also nice and i'll ...",littile big,1446249600,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,[Children will be excited to get active in fas...,$9.95 - $24.99,"[B07D6MJRRL, B015SSQCO6, B01HO66CPE, B00CHNS6Y...","[B07D6MJRRL, B00U9TWIJS, B07CS9VTPB, B00CHNS6Y...","class=""a-normal a-align-center a-spacing-smal...","\n <div class=""content"">\n\n\n\n\n\n\n<ul...",NaN,NaN
3251,5,True,"09 21, 2015",A24YEGDNPYJGO,B00VJX7AC8,yamilia olivera rivera,Thank you,Five Stars,1442793600,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,[Manchester United Home Jersey 2015 / 2016],$43.56

In [54]:
DEBUG = False
SENTI_MODEL = "spacy"

for index in range(len(reduced_review_with_meta)):
    (brand, product, review, rating) = (str(reduced_review_with_meta.iloc[index]["brand"]), 
                                        str(reduced_review_with_meta.iloc[index]["title"]), 
                                        str(reduced_review_with_meta.iloc[index]["reviewText"]),
                                        str(reduced_review_with_meta.iloc[index]["overall"]))

    print(f"Index: {index}")
    print(f"Brand: {brand}")
    print(f"Product: {product}")
    print(f"Review Text: {review}")
    print(f"Product Rating: {rating}")
    adsa_model(review, DEBUG=DEBUG, SENTI_MODEL=SENTI_MODEL)

Index: 0
Brand: adidas
Product: adidas Boys' Triple Up Short
Review Text: Good
Product Rating: 5
Review: Good
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Index: 1
Brand: adidas
Product: adidas Boys' Triple Up Short
Review Text: Perfect fit
Product Rating: 5
Review: Perfect fit
Topic_prediction: {'size': 'Positive', 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Index: 2
Brand: adidas
Product: adidas Boys' Triple Up Short
Review Text: works great. My son likes them.
Product Rating: 5
Review: works great. My son likes them.
Topic_prediction: {'size': 'Positive', 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Index: 3
Brand: adidas
Product: adi

Review: This t-shirt is a longer style which is good for a bit more coverage over the hips. The fabric is nice and light and a great colour.
Topic_prediction: {'size': 'Positive', 'comfort': None, 'appearance': 'Positive', 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Index: 16
Brand: adidas
Product: adidas Women's Ultimate Workout Short-Sleeve V-Neck Top
Review Text: Great fabric, soft, adjust to your shape. If you don't like it to tight buy next size.
I will buy more.
Product Rating: 5
Review: Great fabric, soft, adjust to your shape. If you don't like it to tight buy next size.
I will buy more.
Topic_prediction: {'size': 'Positive', 'comfort': 'Positive', 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Index: 17
Brand: adidas
Product: adidas Women's Ultimate Workout Short-Sleeve V-Neck Top
Review Text: el producto es excelente lo

Review: My son LOVES Adidas.  He's 15 years old, 5'3  and skinny (about 95 lbs.).  He likes his shorts to cover his knees when standing so that they don't ride up too high when sitting.  These shorts fit the bill in XL.  They are a tad wide for him but he doesn't mind, thanks to the drawstring in the elastic waist and the flowing quality of the fabric.
Topic_prediction: {'size': 'Positive', 'comfort': None, 'appearance': None, 'quality': 'Positive', 'price': None, 'delivery': None}
---------------------------------------------------------------

Index: 32
Brand: adidas
Product: adidas Boys' Triple Up Short
Review Text: Fit great! Great buy!
Product Rating: 5
Review: Fit great! Great buy!
Topic_prediction: {'size': 'Positive', 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Index: 33
Brand: adidas
Product: adidas Boys' Triple Up Short
Review Text: soso
Product Rating: 3
Review: soso
T

In [55]:
DEBUG = True
SENTI_MODEL = "spacy"
review = "The dress is very pretty, but it runs very small in the waist and very large in the bust. If you're not an extreme hour-glass, be prepared to alter this before wearing it. I'm short, so I plan to take some fabric either from the hem or the oversized bust to redistribute as needed, personally. Not as comfortable as some of the other items I've gotten from SheIn before - the fabric is a little scratchier than anticipated - but it does flow nicely when you move around."

adsa_model(review, DEBUG=DEBUG, SENTI_MODEL=SENTI_MODEL)

Sentence 1: The dress is very pretty, but it runs very small in the waist and very large in the bust.


Adjectives: [pretty, small, large]
Nouns: [dress, waist, bust]
Pronouns: [it]
Negations: []

Trace: Indirect Reference Detected for Adjective: pretty
Descriptor_pair: ('very pretty', dress, False, 'appearance', 'Positive')

Trace: Indirect Reference Detected for Adjective: small
Descriptor_pair: ('very small', it, False, 'size', 'Negative')

Trace: Direct Reference Detected for Adjective: large
Descriptor_pair: ('very large', waist, False, 'size', 'Negative')

Sentence 2: If you're not an extreme hour-glass, be prepared to alter this before wearing it.


Adjectives: [extreme]
Nouns: [hour, glass]
Pronouns: [you, it]
Negations: [not]

Trace: Direct Reference Detected for Adjective: extreme

Sentence 3: I'm short, so I plan to take some fabric either from the hem or the oversized bust to redistribute as needed, personally.


Adjectives: [short, oversized]
Nouns: [fabric, bust]
Pronouns: [I, I]
Negations: []

Trace: Indirect Reference Detected for Adjective: short
Descriptor_pair: ('short', I, False, 'size', 'Negative')

Trace: Direct Reference Detected for Adjective: oversized

Sentence 4: Not as comfortable as some of the other items I've gotten from SheIn before - the fabric is a little scratchier than anticipated -


Adjectives: [comfortable, other, little, scratchier]
Nouns: [items, fabric]
Pronouns: [I]
Negations: [Not]

Trace: Finding noun subject for Adjective: comfortable
Matching adjective without noun for Adjective: comfortable
Descriptor_pair: ('as comfortable', None, False, 'comfort', 'Positive')

Trace: Direct Reference Detected for Adjective: other

Trace: Finding noun subject for Adjective: little
Matching adjective without noun for Adjective: little
Descriptor_pair: ('little', None, False, 'size', 'Positive')

Trace: Indirect Reference Detected for Adjective: scratchier
Descriptor_pair: ('scratchier', the fabric, False, 'quality', 'Positive')

Sentence 5: but it does flow nicely when you move around.


Adjectives: []
Nouns: []
Pronouns: [it, you]
Negations: []

All descriptor_pair: [('very pretty', dress, False, 'appearance', 'Positive'), ('very small', it, False, 'size', 'Negative'), ('very large', waist, False, 'size', 'Negative'), ('short', I, False, 'size', 'Negative'), ('as comfortable', None, False, 'comfort', 'Positive'), ('little', None, False, 'size', 'Positive'), ('scratchier', the fabric, False, 'quality', 'Positive')]

Review: The dress is very pretty, but it runs very small in the waist and very large in the bust. If you're not an extreme hour-glass, be prepared to alter this before wearing it. I'm short, so I plan to take some fabric either from the hem or the oversized bust to redistribute as needed, personally. Not as comfortable as some of the other items I've gotten from SheIn before - the fabric is a little scratchier than anticipated - but it does flow nicely when you move around.
Topic_prediction: {'size': 'Negative', 'comfort': 'Positive', 'appearance': 'Positiv

{'size': 'Negative',
 'comfort': 'Positive',
 'appearance': 'Positive',
 'quality': 'Positive',
 'price': None,
 'delivery': None}